# Raster masks from vector masks

[xsar.Sentinel1Dataset.dataset](../basic_api.rst#xsar.Sentinel1Dataset.dataset) has a `land_mask` variable by default, rasterized from [`cartopy.feature.NaturalEarthFeature('physical', 'land', '10m')`](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html#cartopy.feature.NaturalEarthFeature)



## Mask sigma0 over land

Replacing `sigma0` values with `nan` over land can be done with `xarray.where`

In [ ]:
import xarray as xr
import xsar
import numpy as np
import os
ds = xsar.open_dataset(xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE'))
ds['sigma0_ocean'] = xr.where(ds['land_mask'], np.nan, ds['sigma0'])
ds['sigma0_ocean']

## Adding masks

Masks can be added with [xsar.Sentinel1Meta.set_mask_feature](../basic_api.rst#xsar.Sentinel1Meta.set_mask_feature), providing a shapefile or a [cartopy.feature.Feature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html) object.

Here, we add a `ocean` mask:

In [ ]:
import cartopy
s1meta = xsar.Sentinel1Meta(xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE'))
s1meta.set_mask_feature('ocean', cartopy.feature.NaturalEarthFeature('physical', 'ocean', '10m'))

Here, we change the default 'land' mask for a high resolution shapefile from openstreetmap

In [ ]:
s1meta.set_mask_feature('land', os.path.join(xsar.get_test_file('land-polygons-split-4326'),'land_polygons.shp'))

Masks are available as a shapely object (lon/lat coordinates), with [xsar.Sentinel1Meta.get_mask](../basic_api.rst#xsar.Sentinel1Meta.get_mask):

In [ ]:
s1meta.get_mask('ocean')

When using this `s1meta` object with  [xsar.Sentinel1Meta.footprint](../basic_api.rst#xsar.Sentinel1Meta.footprint), mask are rasterized over the dataset, and variables postfixed with `_mask` are created.

In [ ]:
del ds
ds = xsar.open_dataset(s1meta)
ds['ocean_mask']

## Convert a mask to dataset coordinates

[xsar.Sentinel1Meta.ll2coords](../basic_api.rst#xsar.Sentinel1Meta.ll2coords) allow converting lon/lat coordinates on shapely objects.

So it's possible to plot the vector mask on a raster variable:

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
hv.extension('bokeh')
gv.extension('bokeh')
from holoviews.operation.datashader import datashade,rasterize

land_poly = s1meta.get_mask('land')
land_poly_coords = s1meta.ll2coords(s1meta.get_mask('land'))

land_poly_coords
hv.Path(land_poly_coords).opts(color='lightgreen',width=800,height=800) 

ds['sigma0_ocean'] = xr.where(ds['land_mask'], np.nan, ds['sigma0'])

rasterize(hv.Image(((ds['sigma0_ocean'].sel(pol='VV')))).opts(cmap='gray',clim=(0,0.05), colorbar=True,tools=['hover'],title="xsar")) \
* hv.Path(land_poly_coords).opts(color='lightgreen',width=800,height=800) 

## Performance comparison

In [ ]:
# Following code is to compare performances for different methods: it's not usefull for the end user
import pandas as pd
import datetime
index = ['default', 'ne_10m_land', 'gshhs_full_land' , 'osm']
columns = ['feature_f', 'feature_init_time', 'land_footprint_time', 'land_footprint_ax_time', 'rasterize_time']

land_features_df = pd.DataFrame(index=index, columns=columns)
land_features_df.loc['default', 'feature_f'] = lambda: cartopy.feature.LAND
land_features_df.loc['ne_10m_land', 'feature_f'] = lambda: cartopy.feature.NaturalEarthFeature('physical', 'land', '10m')
land_features_df.loc['gshhs_full_land', 'feature_f'] = lambda: cartopy.feature.GSHHSFeature(scale='full')
land_features_df.loc['osm', 'feature_f'] = lambda: os.path.join(xsar.get_test_file('land-polygons-split-4326'),'land_polygons.shp')

raster_shape = s1meta.rio.shape
for feature_idx, feature_rows in land_features_df.iterrows():
    t1 = datetime.datetime.now()
    s1meta.set_mask_feature('land',feature_rows['feature_f']())
    land_features_df.at[feature_idx, 'feature_init_time'] = datetime.datetime.now() - t1
    t1 = datetime.datetime.now()
    land_footprint = s1meta.get_mask('land')
    land_features_df.at[feature_idx, 'land_footprint_time'] = datetime.datetime.now() - t1
    t1 = datetime.datetime.now()
    land_footprint_ax = s1meta.ll2coords(land_footprint)
    land_features_df.at[feature_idx, 'land_footprint_ax_time'] = datetime.datetime.now() - t1
    ds = xsar.open_dataset(s1meta)
    t1 = datetime.datetime.now()
    raster = ds['land_mask'].persist()
    del raster
    land_features_df.at[feature_idx, 'rasterize_time'] = datetime.datetime.now() - t1
land_features_df
